# Code tutorials for Runge-Kutta Methods, part I: Implementation of explicit RKM

$\newcommand{mb}[1]{\mathbf{#1}}$
If you want to have a nicer theme for your jupyter notebook, download the cascade stylesheet file calculus4N.css and execute the next cell:

In [1]:
from IPython.core.display import HTML
def css_styling():
    try:
        fname = "calculus4N.css"
        with open(fname, "r") as f:
            styles = f.read()
            return HTML(styles)
    except FileNotFoundError:
        print(f"Could not find css file '{fname}'")

# Comment out next line and execute this cell to restore the default notebook style 
css_styling()

As always, we start by calling the necessary modules: And of course we want to import the required modules.

In [3]:
%matplotlib widget

import ipywidgets as widgets
from ipywidgets import interact, fixed
import numpy as np
from numpy import pi
from numpy.linalg import solve, norm    
import matplotlib.pyplot as plt

# Use a funny plotting style
plt.xkcd()

newparams = {'figure.figsize': (6.0, 6.0),
             'axes.grid': True,
             'lines.markersize': 8, 
             'lines.linewidth': 2,
             'font.size': 14}
plt.rcParams.update(newparams)

## Some programming basics: classes in Python

## Plan
Start from simplest dog class to explain 
  * class vs instance variables, special `__init__` function, 
  * meaning of `self` argument, 
  * add bark/fetch stick functions
  * Write ExplicitRungeKutta class with a `solve` function
  * Introduce special class function `__call__`

In [4]:
# Let's develop a simple dog class

## Implementation of Runge-Kutta methods

### Review:  Runge-Kutta methods

**Explicit** Runge-Kutta schemes are specified in the form of a **Butcher table**:

\begin{equation*}
\begin{array}{c|ccc}
c_1 & a_{11} & \cdots & a_{1s}
\\ 
\vdots & \vdots & & \vdots
\\ 
c_s & a_{s1} & \cdots & a_{ss}
\\ 
\hline
& b_1 & \cdots & b_s
\end{array}
\end{equation*}

with $c_1 = 0$ and $a_{ij} = 0$ for $j \geqslant i$.

So starting from $y_i, t_i$ and chosen step size $\tau_i$
the discrete solution at $t_{i+1}$ is computed as follows

* Compute stage derivatives $k_j$ for $j=1,\ldots,s$:
  \begin{equation*}
  k_{j} =
  f(t_i + c_j \tau, y_i +  \tau \sum_{l=1}^{j-1} {a}_{jl} k_l)
  \end{equation*}
* Compute the next time step via
  \begin{equation*}  
  y_{i+1} = y_{i} + \tau \sum_{j=1}^s b_j k_j
  \end{equation*}

In [ ]:
class ExplicitRungeKuttaAlt:
    def __init__(self, a, b, c):
        self.a = a
        self.b = b
        self.c = c
        
    def solve(self, y0, t0, T, f, Nmax):
        
        # Extract Butcher table
        ..
        
        # Initiate stages
        s = len(b)
        ks = ...

        # Start time-stepping
        ...
        
        while(ts[-1] < T):
            t, y = ts[-1], ys[-1]
            
            # Compute stages derivatives k_j
            for j in range(s):
                ...
                
            # Compute next time-step by computing the incremement dy
            dy = np.zeros_like(y, dtype=np.double)
            ...
            
            ys.append(y + dt*dy)
            ts.append(t + dt)
            
        return (np.array(ts), np.array(ys))

We recall the `compute_eoc` function.

In [ ]:
def compute_eoc(y0, t0, T, f, Nmax_list, solver, y_ex):
    errs = [ ]
    for Nmax in Nmax_list:
        ts, ys = solver(y0, t0, T, f, Nmax)
        ys_ex = y_ex(ts)
        errs.append(np.abs(ys - ys_ex).max())
        print("For Nmax = {:3}, max ||y(t_i) - y_i||= {:.3e}".format(Nmax,errs[-1]))

    errs = np.array(errs)
    Nmax_list = np.array(Nmax_list)
    dts = (T-t0)/Nmax_list

    eocs = np.log(errs[1:]/errs[:-1])/np.log(dts[1:]/dts[:-1])

    # Insert inf at beginning of eoc such that errs and eoc have same length 
    eocs = np.insert(eocs, 0, np.Inf)

    return errs, eocs

### Task: Check your implementation

Test your implementation for 2 different Butcher tables of your choice. For each of Runge-Kutta methods, consider two test problems

* $y'= \lambda y$, $y(t_0) = y_0$ with exact solution $y(t) = y_0 e^{\lambda (t-t_0)}$

* $y' = -2 t y$,  $y(0) = y_0$  with exact solution $y(t) = e^{-t^2}$

1. you can do a quick run and solve these test problems numerical for some $N_{max}$ to check whether you get reasonable results.

2. If this is the case, you can do a mor thorough test by checking the EOC using the `compute_eoc` function above.
